# Plotting Transfer Functions

**Roberto dos Reis**<br>
Northwestern University


Calculate CTEM bright field phase contrast transfer function with partial coherence for weak phase objects


**References:** <br>
R.H. Wade, J. Frank, Optik, 49 (1977), p. 81
Ishizuka, K., 1980. Contrast transfer of crystal images in TEM. Ultramicroscopy, 5(1-3), pp.55-65.

### Load necessary packages:

In [5]:
# Numeric operations
import numpy as np

# Visualization / Plotting
import matplotlib
from matplotlib import pyplot as plt
from matplotlib.patches import Rectangle
import seaborn as sns

# for math functions
import scipy.special as sp
from scipy.interpolate import interp1d
# for complex numbers operations
import cmath 
import math 


# Ensure that images are rendered in this notebook:
%matplotlib inline

## Input microscope parameters

In [ ]:
def ctemh(k,params, type):
    Cs3 = params[0]*1.0e7
    Cs5 = params[1]*1.0e7
    df = params[2]
    kev = params[3]
    ddf = params[4]
    beta = params[5]*0.001
    mo = 511.0 # electron rest mass in keV
    hc = 12.3986 # in keV-Angstroms
    wav = (2*mo)+kev;
    wav = hc/np.sqrt(wav*kev)
    wavsq = wav*wav;
    
    w1 = np.pi*Cs3*wavsq*wav
    w2 = np.pi*wav*df;
    w3 = np.pi*Cs5*wavsq*wavsq*wav
    e0 = (np.pi*beta*ddf)**2
    k2 = k*k
    
    wr = ((w3*k2+w1)*k2-w2)*k*beta/wav
    wi = np.pi*wav*ddf*k2
    wi = wr*wr + 0.25*wi*wi
    wi = np.exp(-wi/(1+e0*k2))
    wr = w3*(1-2.0*e0*k2)/3.0
    wr = wr*k2 + 0.5*w1*(1-e0*k2)
    wr = (wr*k2 - w2)*k2/(1+e0*k2);
    if type == 0:
        y = np.sin(wr)* wi
    else:
        y = np.cos(wr)* wi
        
    return y

### Calculate CTEM Transfer functions

Cs3,5 = Spherical Aberrations<br>
df = defocus<br>
kev = electron energy in keV<br>
ddf = chromatic aberation defocus spread<br>
beta =  spread in illumination angles<br>

In [ ]:
print( 'Plot CTEM transfer function' )
kev= float(input( 'Type electron energy in keV :'))
Cs3= float(input( 'Type spherical aberation Cs3 in mm :'))
Cs5= float(input( 'Type spherical aberation Cs5 in mm :'))
df= float(input( 'Type defocus df in Angstroms :'))
ddf= float(input( 'Type defocus spread ddf in Angstroms :'))
beta= float(input( 'Type illumination semiangle in mrad :'))
type= float(input( 'Type 0 for phase contrast, 1 for amplitude:' ))

In [ ]:
# electron wavelength
wav = 12.3986/np.sqrt((2*511.0 + kev)*kev);
ds = np.sqrt(np.sqrt(Cs3*1.0e7*wav*wav*wav ));

kmax = 2.5/ds;
k = np.linspace(0, kmax, num=500)

params = [Cs3, Cs5, df, kev, ddf, beta ];
sinw = ctemh(k, params,type);

In [ ]:
fig, ax = plt.subplots(figsize=(6, 4))
plt_handle = ax.plot(k,sinw, 'k')
ax.set_title(f'E = {params[3]} keV, Cs3 = {params[0]} mm, Cs5={params[3]} mm, df = {params[2]} A'
             +"\n"+
            f'Beta= {params[5]}mrad,ddf={params[3]}A, fontSize = 12')

ax.set_ylabel(r'MTF', fontsize=15)
ax.set_xlabel(r'Spatial Frequency', fontsize=15)
# ax.set_title('Volume and percent change')

ax.grid(True)
fig.tight_layout()
plt.show()

### Calculate STEM Probe Profile

Cs3,5 = Spherical Aberrations<br>
df = defocus<br>
kev = electron energy in keV<br>
ddf = chromatic aberation defocus spread<br>
beta =  spread in illumination angles<br>

In [59]:
pkev = float(input( 'Type electron energy in keV : '))
pCs3 = abs(float(input( 'Type spherical aberration Cs3 in mm : ')))
pCs5 = float(input( 'Type spherical aberration Cs5 in mm : '))
pdf = float(input( 'Type defocus df in Angstroms : '))
pamax = float(input( 'Type obj. apert. semiangle in mrad : '))
pd0 = float(input('Type source size in Angstroms: '))

Type electron energy in keV : 200
Type spherical aberration Cs3 in mm : 1.3
Type spherical aberration Cs5 in mm : 0
Type defocus df in Angstroms : 700
Type obj. apert. semiangle in mrad : 10
Type source size in Angstroms: 1


In [60]:
probe_params = [pkev, pCs3, pCs5, pdf, pamax, pd0];

In [69]:
def stemhr(r,params,nk):
    P2 = 2*np.pi
    kev = params[0]
    Cs3 = params[1]*1.0e7
    Cs5 = params[2]*1.0e7
    df =  params[3]
    amax = params[4]*0.001
    d0 = params[5]
    
    #electron wavelength
    wav = 12.3986/np.sqrt((2*511.0+kev)*kev)
    kmax = amax/wav
    dk = kmax/nk
    k = np.linspace(0, kmax, num=nk)
    k2 = k*k
    w1 = 0.5*Cs*wav*wav*wav
    w2 = wav*df
    w = np.pi*(w1*k2-w2 )*k2
    expw = np.exp(-1j*w )
    
    nr = len(r)
    dr = r[nr-1]/nr
    psf = np.zeros((nr,), dtype=int)
    #tabulate besselj0 on a grid 5 times finer than nk
    bmax = 2*np.pi*r[nr-1]*kmax
    db = bmax/(5*nk)
    b = np.linspace(0, bmax, num=int(bmax/db))
    bessj0 = sp.jv(0,b)
    
    for ir in range(1, nr):
#         P2*r[ir-1]*k
        h = expw*interp1d(b,bessj0,'cubic')*k
        psf[ir] = np.abs(np.sum(h)**2)*2/(nk*np.pi)
            
    psf[nr-1]=0
    k = np.linspace(0,2*kmax, num = int(kmax/dk))
    nk = len(k)
    
    for ik in range(0, nk):
#         P2*r*k[ik]
        h = psf.transpose *interp1d(b,bessj0,'cubic')*r
        mtf[ik]=np.sum(h)
        # damp by source spread
                
    mtf = mtf*np.exp(-0.5*(k*d0/P2)**2)
    
    #transform mtf back to psf 
    for ir in range(1, nr):
#         P2*r[ir-1]*k
        h = mtf*interp1d(b,bessj0,'cubic')*k
        psf[ir-1]= np.sum(h)/(nk*np.pi)
        psf[nr-1]=0
                    
    nrm = 2*np.pi*np.sum(psf*r.transpose)*dr
    psf = psf/nrm
    
    return psf

In [70]:
wav = 12.3986/np.sqrt((2*511.0+probe_params[0])*probe_params[0]);
# Cs = abs(probe_params[1])

if(Cs<0.1):
    Cs = 0.1
    end
    
rmax = np.sqrt(np.sqrt(Cs*1.0e7*wav*wav*wav))

npts=300 #number of points in curve
r=np.linspace(0,rmax,num=npts)

#Calling stemhr function here
psf = stemhr(r,probe_params,npts);

TypeError: unsupported operand type(s) for *: 'complex' and 'interp1d'

In [ ]:
plot(r,psf);
xlabel('radius in Angstroms');
ylabel('PSF');

In [ ]:
plt.scatter(k,wr)

## Reading file containing atomic positions

Create atomic coordinates for use in multislice simulation.

In [ ]:
cellDim = [26.877129, 27.15, 5.43]

filename = 'Si_7x5.dat'
atoms = np.loadtxt(filename, delimiter=',')

print (atoms)

In [ ]:
# projected potential

In [ ]:
function V = tempot(xmax,ymax,nx,ny,potfile)

%  TEMPOT Generate a Projected Potential suitable for TEM,  i.e intensity is proportional to Z^1/3
%  V = stempot(xmax,ymax,nx,ny,potfile)
%  inputs xmax, ymax are the size of the slice in angstroms
%  nx,ny are the number of pixels in the x and y directions 
%
%  started 10-Apr-1997 David Muller 
%
zed = 0.67;  % zed=2 for rutherford scattering of the nucleus, less for screening

ix = 1:nx;
iy = 1:ny;
dx = xmax/(nx-1);
dy = ymax/(ny-1);
rx = 0:dx:xmax-dx;
ry = 0:dy:ymax-dy;

if isstr( potfile)  
   A = readxyz( potfile);
else
   A = potfile;   % passed a variable instead
end;

Zatom = A.Znum;
ax    = A.xpos;
ay    = A.ypos;
az    = A.zpos;
wt    = A.wt;
amax = length(Zatom);

% find boundaries of slice
axmin = min(ax);
axmax = max(ax);
aymin = min(ay);
aymax = max(ay);
% shift coords to fit in box
ax = ax - axmin;
ay = ay - aymin;


V= zeros(nx,ny);

% map x and y coords of the atoms to the nearest grid points
% A fraction of the atom must be assigned to the closest gridpoints
% to avoid sum and difference frequencies appearing in the image

iax = max(1,min(floor( ax/dx)+1,nx));      % grid point to the left of the atom
%jax = interp1( rx,ix,ax, 'linear');
ibx = mod(iax,nx) + 1;      % create periodic boundary conditions

fax = 1-mod( (ax /dx ),1 );  % fraction of atom at iax 
%fbx = 1 - fax;            % fraction of atom at (1-fax) is at iax+1

iay = max(1,min(floor( ay/dy)+1,ny));      % grid point above the atom
%jay = interp1( ry,iy,ay, 'linear');

iby = mod(iay,ny) +1;          % create periodic boundary conditions

fay = 1-mod( (ay /dy ),1 ); % fraction of atom at iay 
%fby = 1 - fay;            % fraction of atom at (1-fay) is at iay+1

% Add each atom to the potential grid
% j is too large to makegrid(iax,iay) which would allow us to vectorize V
%

V1 = fax .* fay .* (Zatom .^zed);
V2 = (1-fax) .* fay .* (Zatom .^zed);
V3 = fax .* (1-fay) .* (Zatom .^zed);
V4 = (1-fax) .* (1-fay) .* (Zatom .^zed);

for j=1:amax,
   V(iax(j),iay(j)) = V(iax(j),iay(j)) + V1(j);
   V(ibx(j),iay(j)) = V(ibx(j),iay(j)) + V2(j);
   V(iax(j),iby(j)) = V(iax(j),iby(j)) + V3(j);
   V(ibx(j),iby(j)) = V(ibx(j),iby(j)) + V4(j);
end;

In [ ]:
# TEMcon